In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Dataset/bbc-text.csv')
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [ ]:
kategori = pd.get_dummies(df.category)
df_new = pd.concat([df, kategori], axis=1)
df_new = df_new.drop(columns='category')
max_len =  df_new['text'].map(lambda x: len(x.split())).max()
print(df_new)
print(max_len)

                                                   text  business  ...  sport  tech
0     tv future in the hands of viewers with home th...         0  ...      0     1
1     worldcom boss  left books alone  former worldc...         1  ...      0     0
2     tigers wary of farrell  gamble  leicester say ...         0  ...      1     0
3     yeading face newcastle in fa cup premiership s...         0  ...      1     0
4     ocean s twelve raids box office ocean s twelve...         0  ...      0     0
...                                                 ...       ...  ...    ...   ...
2220  cars pull down us retail figures us retail sal...         1  ...      0     0
2221  kilroy unveils immigration policy ex-chatshow ...         0  ...      0     0
2222  rem announce new glasgow concert us band rem h...         0  ...      0     0
2223  how political squabbles snowball it s become c...         0  ...      0     0
2224  souness delight at euro progress boss graeme s...         0  ...      

In [ ]:
import re
#from nltk.download import stopwords
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

space = re.compile('[/(){}\[\]\|@,;]')
symbols= re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_cleaning(teks):
    teks = teks.lower()
    teks = space.sub(' ', teks)
    teks = symbols.sub('', teks)
    teks = teks.replace('x', '')
    teks = ' '.join(word for word in teks.split() if word not in STOPWORDS)
    return teks
df_new['text']=df_new['text'].apply(text_cleaning)

In [ ]:
max_word = df_new['text']
max_sentence = max_word.map(lambda x: len(x.split())).max()
desc = df_new['text'].values
label = df_new[['business',	'entertainment', 'politics', 'sport',	'tech']].values
print(desc)
print(label)
print(max_sentence)

['tv future hands viewers home theatre systems plasma highdefinition tvs digital video recorders moving living room way people watch tv radically different five years time according epert panel gathered annual consumer electronics show las vegas discuss new technologies impact one favourite pastimes us leading trend programmes content delivered viewers via home networks cable satellite telecoms companies broadband service providers front rooms portable devices one talkedabout technologies ces digital personal video recorders dvr pvr settop boes like us tivo uk sky+ system allow people record store play pause forward wind tv programmes want essentially technology allows much personalised tv also builtin highdefinition tv sets big business japan us slower take europe lack highdefinition programming people forward wind adverts also forget abiding network channel schedules putting together alacarte entertainment us networks cable satellite companies worried means terms advertising revenues

In [ ]:
from sklearn.model_selection import train_test_split
desc_train, desc_test, label_train, label_test = train_test_split(desc, label, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
     
tokenizer = Tokenizer(num_words=40000, oov_token='x')
tokenizer.fit_on_texts(desc_train) 
tokenizer.fit_on_texts(desc_test)
     
seq_train = tokenizer.texts_to_sequences(desc_train)
seq_test = tokenizer.texts_to_sequences(desc_test)
     
pad_train = pad_sequences(seq_train, maxlen=max_sentence) 
pad_test = pad_sequences(seq_test, maxlen=max_sentence)

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=40000, output_dim=150, input_length=max_sentence),
    tf.keras.layers.SpatialDropout1D(0.25),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='RMSProp',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2213, 150)         6000000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 2213, 150)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                55040     
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 6,071,941
Trainable params: 6,071,941
Non-trainable params: 0
______________________________________________

In [ ]:
num_epochs = 4
history = model.fit(pad_train, label_train, epochs=num_epochs,
                    validation_data=(pad_test, label_test), verbose=2)

Epoch 1/4
56/56 - 101s - loss: 1.3534 - accuracy: 0.3685 - val_loss: 1.0102 - val_accuracy: 0.4921
Epoch 2/4
56/56 - 100s - loss: 0.7734 - accuracy: 0.6764 - val_loss: 0.4465 - val_accuracy: 0.9011
Epoch 3/4
56/56 - 100s - loss: 0.2272 - accuracy: 0.9399 - val_loss: 0.3141 - val_accuracy: 0.9213
Epoch 4/4
56/56 - 100s - loss: 0.0655 - accuracy: 0.9826 - val_loss: 0.2878 - val_accuracy: 0.9303
